# Group Metrics

The `fairlearn` package contains algorithms which enable machine learning models to minimise disparity between groups. The `metrics` portion of the package provides the means required to verify that the mitigation algorithms are succeeding.

In [ ]:
import numpy as np
import pandas as pd
import sklearn.metrics as skm

## Ungrouped Metrics

At their simplest, metrics take a set of 'true' values $Y_{true}$ (from the input data) and predicted values $Y_{pred}$ (by applying the model to the input data), and use these to compute a measure. For example, the _recall_ or _true positive rate_ is given by
\begin{equation}
P( Y_{pred}=1 | Y_{true}=1 )
\end{equation}
That is, a measure of whether the model finds all the positive cases in the input data. The `scikit-learn` package implements this in [sklearn.metrics.recall_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html).

Suppose we have the following data:

In [ ]:
Y_true = [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1]
Y_pred = [0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]

we can see that the prediction is 1 in five of the ten cases where the true value is 1, so we expect the recall to be 0.0.5:

In [ ]:
skm.recall_score(Y_true, Y_pred)

## Metrics with Grouping

When considering fairness, each row of input data will have an associated group label $g \in G$, and we will want to know how the metric behaves for each $g$. To help with this, Fairlearn provides wrappers, which take an existing (ungrouped) metric function, and apply it to each group within a set of data.

Suppose in addition to the $Y_{true}$ and $Y_{pred}$ above, we had the following set of labels:

In [ ]:
group_membership_data = ['d', 'a', 'c', 'b', 'b', 'c', 'c', 'c', 'b', 'd', 'c', 'a', 'b', 'd', 'c', 'c']

df = pd.DataFrame({ 'Y_true': Y_true, 'Y_pred': Y_pred, 'group_membership_data': group_membership_data})
df

In [ ]:
import fairlearn.metrics as flm

group_metrics = flm.group_summary(skm.recall_score, Y_true, Y_pred, sensitive_features=group_membership_data, sample_weight=None)

print("Overall recall = ", group_metrics.overall)
print("recall by groups = ", group_metrics.by_group)

Note that the overall recall is the same as that calculated above in the Ungrouped Metric section, while the `by_group` dictionary matches the values we calculated by inspection from the table above.

In addition to these basic scores, `fairlearn.metrics` also provides convenience functions to recover the maximum and minimum values of the metric across groups and also the difference and ratio between the maximum and minimum:

In [ ]:
print("min recall over groups = ", flm.group_min_from_summary(group_metrics))
print("max recall over groups = ", flm.group_max_from_summary(group_metrics))
print("difference in recall = ", flm.difference_from_summary(group_metrics))
print("ratio in recall = ", flm.ratio_from_summary(group_metrics))

## Supported Ungrouped Metrics

To be used by `group_summary`, the supplied Python function must take arguments of `y_true` and `y_pred`:
```python
my_metric_func(y_true, y_pred)
```
An additional argument of `sample_weight` is also supported:
```python
my_metric_with_weight(y_true, y_pred, sample_weight=None)
```
The `sample_weight` argument is always invoked by name, and _only_ if the user supplies a `sample_weight` argument.

## Convenience Wrapper

Rather than require a call to `group_summary` each time, we also provide a function which turns an ungrouped metric into a grouped one. This is called `make_metric_group_summary`:

In [ ]:
recall_score_group_summary = flm.make_metric_group_summary(skm.recall_score)

results = recall_score_group_summary(Y_true, Y_pred, sensitive_features=group_membership_data)

print("Overall recall = ", results.overall)
print("recall by groups = ", results.by_group)